# Real Estate Regression
### Building a Regression Model to Predict Housing Prices in NY City 
### Jackson Patrick & Trevor Chartier

## Introduction

## Part 1: Data Processing

In [767]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from geopy.distance import geodesic # may need to run pip install geopy
import pandas.api.types as pd_types

### Read In Dataset

In [768]:

house_df = pd.read_csv("data/NY-House-Dataset.csv")
print(house_df.shape[0], " samples, ", house_df.shape[1], " features")
house_df.head()

4801  samples,  17  features


,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000,1400.000,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761,-73.974
1,Brokered by Serhant,Condo for sale,195000000,7,10.000,17545.000,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766,-73.981
2,Brokered by Sowae Corp,House for sale,260000,4,2.000,2015.000,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.542,-74.196
3,Brokered by COMPASS,Condo for sale,69000,3,1.000,445.000,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761,-73.975
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.374,14175.000,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767,-73.970


### Feature Engineering: Distance to Nearest Subway Station

Before analyzing the dataset and it's features, we would like to add one additional feature based on longitude and latitude -- the distance from the property to the nearest subway station.

In [769]:
subway_df = pd.read_csv("data/MTA_Subway_Stations.csv")
subway_df = subway_df[["GTFS Latitude", "GTFS Longitude"]]
subway_df.rename(columns={"GTFS Latitude": "Latitude", "GTFS Longitude":"Longitude"}, inplace=True)
print(subway_df.shape[0], " Subway Stations Included in NYC Dataset")
subway_df.head()

496  Subway Stations Included in NYC Dataset


,Latitude,Longitude
0,40.726,-73.995
1,40.589,-73.984
2,40.834,-73.851
3,40.681,-73.956
4,40.718,-73.987


In [770]:
subway_coords = subway_df[['Latitude','Longitude']].values
subway_tree = cKDTree(subway_coords)

subway_distances = []

for _, property in house_df.iterrows():
    property_coord = (property["LATITUDE"], property["LONGITUDE"])
    
    distance, idx = subway_tree.query([property["LATITUDE"], property["LONGITUDE"]])
    
    nearest_subway_coord = subway_coords[idx]
    precise_distance = np.round(geodesic(property_coord, nearest_subway_coord).miles,6)
    subway_distances.append(precise_distance)
    
house_df["NEAREST_SUB_STATION"] = subway_distances

In [771]:
house_df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,NEAREST_SUB_STATION
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000,1400.000,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761,-73.974,0.085
1,Brokered by Serhant,Condo for sale,195000000,7,10.000,17545.000,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766,-73.981,0.121
2,Brokered by Sowae Corp,House for sale,260000,4,2.000,2015.000,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.542,-74.196,0.605
3,Brokered by COMPASS,Condo for sale,69000,3,1.000,445.000,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761,-73.975,0.091
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.374,14175.000,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767,-73.970,0.247


In [772]:
print(len(list(house_df.columns)), "total features after feature engineering")
print("Features: ", list(house_df.columns))

18 total features after feature engineering
Features:  ['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'ADDRESS', 'STATE', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'LOCALITY', 'SUBLOCALITY', 'STREET_NAME', 'LONG_NAME', 'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE', 'NEAREST_SUB_STATION']


### Analysis of Numerical Variables

In [773]:
# Defining a function to safely drop columns (in case certain cells get run twice)
def drop_column(column_name):
    if column_name in house_df.columns:
        house_df.drop(column_name, axis=1, inplace=True)

#### Missing Values

In [774]:
# It appears that in this dataset, missing values are filled in with the mean value for the column
# We want to identify columns with potentially large numbers of missing values

def num_filled(colName):
    mean_val = np.mean(house_df[colName])
    is_close = np.isclose(house_df[colName], mean_val, atol=1e-9)
    return sum(is_close)

for column in house_df:
    if(pd_types.is_numeric_dtype(house_df[column])):
        print(num_filled(column), "instances in", column, " have been filled with the mean value of the column")

0 instances in PRICE  have been filled with the mean value of the column
0 instances in BEDS  have been filled with the mean value of the column
302 instances in BATH  have been filled with the mean value of the column
1621 instances in PROPERTYSQFT  have been filled with the mean value of the column
21 instances in LATITUDE  have been filled with the mean value of the column
28 instances in LONGITUDE  have been filled with the mean value of the column
0 instances in NEAREST_SUB_STATION  have been filled with the mean value of the column


### Analysis of Categorical Features

Categorical features with very high cardinality, warrant inspection for the following reasons:
* Insufficient number of samples per category
* Increase of computational complexity to encode
* Lead to overfitting if model memorizes categories rather than learning patterns

In [775]:
categorical_columns = ['BROKERTITLE', 'TYPE', 'ADDRESS', 'STATE', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'LOCALITY', 'SUBLOCALITY', 'STREET_NAME', 'LONG_NAME', 'FORMATTED_ADDRESS']
cardinalities = []

for column in categorical_columns:
    cardinalities.append(len(house_df[column].unique()))
    
df = pd.DataFrame({"Categorical Feature" :categorical_columns, "Cardinality":cardinalities})
df

,Categorical Feature,Cardinality
0,BROKERTITLE,1036
1,TYPE,13
2,ADDRESS,4582
3,STATE,308
4,MAIN_ADDRESS,4583
5,ADMINISTRATIVE_AREA_LEVEL_2,29
6,LOCALITY,11
7,SUBLOCALITY,21
8,STREET_NAME,174
9,LONG_NAME,2731


#### BROKERTITLE

In [776]:
broker_counts = house_df["BROKERTITLE"].value_counts()
threshold = len(house_df) * .01 # Determine a "rare" category to be one with fewer than 1% of total examples
print(len(broker_counts[broker_counts > threshold]), "brokers have at least 1% of total samples")
print("Together, these 10 brokers represent ", broker_counts[broker_counts > threshold].sum(), "samples")
print("\nNumber of properties brokered by various brokers: ")
broker_counts

10 brokers have at least 1% of total samples
Together, these 10 brokers represent  1124 samples

Number of properties brokered by various brokers: 


Brokered by COMPASS                              456
Brokered by Douglas Elliman - 575 Madison Ave    110
Brokered by Brown Harris Stevens                  93
Brokered by Corcoran East Side                    91
Brokered by RE MAX Edge                           79
                                                ... 
Brokered by Gerard R Desgranges                    1
Brokered by Property Professional Realty           1
Brokered by Mark Murphy Properties                 1
Brokered by S Sharf Realty Inc                     1
Brokered by New York Way Real Estate Corp          1
Name: BROKERTITLE, Length: 1036, dtype: int64

There are some brokers with a significant number of samples in the dataset. Specifically, 10 brokers have at least 49 representative samples (1% of total). However, the vast majority of BROKERTITLE categories are rare (ie contain less than 1%) and many contain just a couple examples.

In [777]:
property_broker_counts = house_df["BROKERTITLE"].map(broker_counts)

print("Pearson Correlation: ", np.round(property_broker_counts.corr(house_df["PRICE"], method="pearson"),3))
print("Spearmen Correlation: ", np.round(property_broker_counts.corr(house_df["PRICE"], method="spearman"),3))
print("Kendall Correlation: ", np.round(property_broker_counts.corr(house_df["PRICE"], method="kendall"),3))


Pearson Correlation:  0.008
Spearmen Correlation:  0.233
Kendall Correlation:  0.159


The list property_broker_counts is a frequency encoding of BROKERTITLE. The low pearson correlation coefficient suggests no linear relationship between property_broker_counts and PRICE. However, the moderate values of Spearmen and Kendall correlation suggest some monotonic relationship between the number of properties sold by a broker and the PRICE. As one increases, so too does the other, but not at a constant rate. There seems to be some signal contained within the BROKERTITLE variable.

The high cardinality of BROKERTITLE, the large number of rare categories, and the seemingly ordinal relationship have led us to a few possibilities:
1. Group categories with fewer than 1% of representative samples into an "other" category
2. Use a frequency encoding
3. Encode BROKERTITLE into a BROKER_ACTIVITY feature containing the categories low activity, medium activity, and high activity

Option 1 will result in 11 total categories and rougly 3/4 of the data belonging to the "other" category. 
Option 2 will also result in a skewed distribution

Our judgement suggests that option 3 may be the most balanced solution that simplifies the feature's cardinality while maintaining the ordinal relationship, and limits highly skewed distributions.

In [778]:
if "BROKER_ACTIVITY" not in house_df.columns:
    broker_counts[broker_counts < 7] = 0 # Brokers with fewer than 7 examples are considered low
    broker_counts[(broker_counts <= 25) & (broker_counts >= 7)] = 1 # Brokers with between 7 and 25 are coonsidered medium
    broker_counts[broker_counts > 25] = 2 # Brokers with more than 25 examples are considered high activity
    
    house_df["BROKER_ACTIVITY"] = house_df["BROKERTITLE"].map(broker_counts)
    house_df.drop("BROKERTITLE", axis=1, inplace=True)

# Output distribution of categories
df = pd.DataFrame(house_df["BROKER_ACTIVITY"].value_counts())
df.reset_index(inplace=True)
df.columns = ["BROKER_ACTIVITY", "Number of Samples"]
df = df.sort_values(by="BROKER_ACTIVITY")
df["Activity Level"] = ['Low', 'Medium', 'High']
df = df[["Activity Level", "BROKER_ACTIVITY", "Number of Samples"]]
df



,Activity Level,BROKER_ACTIVITY,Number of Samples
1,Low,0,1708
2,Medium,1,1217
0,High,2,1876


Admittedly, the cutoff points of 7 and 25 were somewhat arbitrary; however, we did try various values before finding ones that we were satisfied with the resulting class distribution. An alternative approach would have been to use values that perfectly divided the classes (or as close as possible), but we felt this was unecessary.

#### Type

In [779]:
type = house_df["TYPE"]
type_values = type.value_counts(normalize=True)
df = type_values.reset_index()
df.columns= ["TYPE", "Proportion of Samples"]
df

,TYPE,Proportion of Samples
0,Co-op for sale,0.302
1,House for sale,0.211
2,Condo for sale,0.186
3,Multi-family home for sale,0.151
4,Townhouse for sale,0.062
5,Pending,0.051
6,Contingent,0.018
7,Land for sale,0.010
8,For sale,0.004
9,Foreclosure,0.003


For TYPE, the cardinality isn't too high; however, there are some categories with an insuficcient number of samples. To handle this feature, I will group categories under the 1% threshold into a singular "other" category.

In [780]:
if "Other" not in house_df["TYPE"].unique(): # Prevent from running the follwoing code twice
    house_df[house_df["TYPE"].map(type_values < .01)] = "Other"
    


#### ADDRESS, MAIN_ADDRESS, FORMATTED_ADDRESS

Not only is there high multicolinearity between these features, but they are nearly unique to each observation, meaning there is nearly no predicitve power within these features. 

They may conatin useful information such as zip code or street name, but this information is also contained within other columns (which we will discuss later)

For these reasons, we have decided to drop these features.

In [781]:
drop_column("ADDRESS")
drop_column("MAIN_ADDRESS")
drop_column("FORMATTED_ADDRESS")

#### STREET_NAME

In [782]:
print(len(house_df["STREET_NAME"].unique()), " unique categories for STREET_NAME")
house_df["STREET_NAME"].head(10)

174  unique categories for STREET_NAME


0    East 55th Street
1            New York
2       Staten Island
3            New York
4            New York
5            Brooklyn
6           Manhattan
7       Staten Island
8     Morrison Avenue
9             Midwood
Name: STREET_NAME, dtype: object

It appears that STREET_NAME was extracted as the second token in the formatted address. Unfortunately, this seems to be an unreliable method to extract the street name and it instead appears to be a mix of street name, county, and state information. We will drop this feature.

In [783]:
drop_column("STREET_NAME")

#### LONG_NAME

In [784]:
print(len(house_df["LONG_NAME"].unique()), " unique categories for LONG_NAME")

house_df["LONG_NAME"].head(10)

2721  unique categories for LONG_NAME


0     Regis Residence
1    West 57th Street
2     Sinclair Avenue
3    East 55th Street
4    East 64th Street
5          Park Place
6                 157
7       Benedict Road
8         Parking lot
9                1350
Name: LONG_NAME, dtype: object

LONG_NAME appears to suffer from the same issues as STREET_NAME, additionally, it has incredibly high cardinality. Due to reliability concerns, this feature will also be dropped.

In [785]:
drop_column("LONG_NAME")

#### SUBLOCALITY

In [786]:
house_df["SUBLOCALITY"].unique()
house_df["SUBLOCALITY"].value_counts().head(20)

New York            999
Kings County        720
Queens County       681
Queens              555
Richmond County     472
Brooklyn            463
Bronx County        306
New York County     282
The Bronx           184
Staten Island        59
Other                42
Manhattan            20
Riverdale             4
Flushing              4
Coney Island          3
East Bronx            1
Brooklyn Heights      1
Jackson Heights       1
Rego Park             1
Fort Hamilton         1
Name: SUBLOCALITY, dtype: int64

SUBLOCALITY contains information on the boroughs and neighborhoods in NYC. I like this concept, especially the bouroughs, as this could capture some information about the socioeconomic status of the broad region of a property. However, after reviewing LONG_NAME and STATE_NAME I am concerned about the overall reliability of this feature and the mix of neighborhoods and bouroughs. 

I will drop this feature and instead use feature engineering from the zip-code to create a feature for bourough. 

In [787]:
drop_column("SUBLOCALITY")

#### LOCALITY

In [788]:
house_df["LOCALITY"].unique()


array(['New York', 'New York County', 'The Bronx', 'Kings County',
       'Bronx County', 'Queens County', 'Richmond County',
       'United States', 'Other', 'Brooklyn', 'Queens', 'Flatbush'],
      dtype=object)

Dropping this column for the same reason as SUBLOCALITY

In [789]:
drop_column("LOCALITY")

#### STATE

In [790]:
state = house_df["STATE"]
print(len(state.unique()), " unique categories for STATE")
state.head()

307  unique categories for STATE


0         New York, NY 10022
1         New York, NY 10019
2    Staten Island, NY 10312
3        Manhattan, NY 10022
4         New York, NY 10065
Name: STATE, dtype: object

STATE is a feature with high cardinality. It also appears that the end of each STATE string is the zip code. Let's focus on this part for now...

In [791]:
# Replace STATE Feature with extracted ZIP_CODE feature
house_df["zip"] = state.apply(lambda x: x.split()[-1])
print(len(house_df["zip"].unique()), "uniqe zip-code categories")

print((house_df["zip"].value_counts(normalize=True) > .01).sum(), " zip-codes have at least 1% of total samples")

179 uniqe zip-code categories
29  zip-codes have at least 1% of total samples


The zip-code may contain valuable information regarding price. To overcoome the sparsity and high cardinality, we will map the zip-codes into the bouroughs of NYC they belong in.

In [792]:
# Load in a mapping from zip code to bourough
zip_borough_df = pd.read_csv("data/zip_borough.csv")
zip_borough_df["zip"] = zip_borough_df["zip"].astype(str)
zip_borough_df.head(3)

,zip,borough
0,10001,Manhattan
1,10002,Manhattan
2,10003,Manhattan


In [793]:
house_df = house_df.merge(zip_borough_df, on="zip", how='left')
house_df['BOROUGH'] = house_df['borough'].fillna('Other')
drop_column('borough')
drop_column("STATE")
drop_column("zip")


#### ADMINISTRATIVE_AREA_LEVEL_2

In [794]:
house_df["ADMINISTRATIVE_AREA_LEVEL_2"].unique()

array(['New York County', 'United States', 'New York', 'Bronx County',
       '11214', '10301', 'Other', '10309', '10303', '11234', '11414',
       '10310', '10003', '11417', '10304', 'Kings County', '10463',
       'Queens County', '10017', '10306', '10471', '11229', '10312',
       '11412', '10465', 'Brooklyn', '10002', '10466', '11237', '11218'],
      dtype=object)

This feature appears to be a mixture of zip-code, county, state, and country values. It appears to be unreliable and contains information already captured elsewhere, such as bourough. We will drop this feature.

In [795]:
drop_column("ADMINISTRATIVE_AREA_LEVEL_2")

In [796]:
house_df.head(20)

,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,LATITUDE,LONGITUDE,NEAREST_SUB_STATION,BROKER_ACTIVITY,BOROUGH
0,Condo for sale,315000,2,2.000,1400.000,40.761,-73.974,0.085,2,Manhattan
1,Condo for sale,195000000,7,10.000,17545.000,40.766,-73.981,0.121,2,Manhattan
2,House for sale,260000,4,2.000,2015.000,40.542,-74.196,0.605,2,Staten
3,Condo for sale,69000,3,1.000,445.000,40.761,-73.975,0.091,2,Manhattan
4,Townhouse for sale,55000000,7,2.374,14175.000,40.767,-73.970,0.247,2,Other
5,House for sale,690000,5,2.000,4004.000,40.674,-73.959,0.064,2,Brooklyn
6,Condo for sale,899500,2,2.000,2184.208,40.809,-73.947,0.135,2,Manhattan
7,House for sale,16800000,8,16.000,33000.000,40.595,-74.106,0.804,0,Staten
8,Co-op for sale,265000,1,1.000,750.000,40.822,-73.874,0.548,0,Bronx
9,Co-op for sale,440000,2,1.000,978.000,40.616,-73.970,0.240,0,Brooklyn
